<a href="https://colab.research.google.com/github/GASKAP/SPARK/blob/master/synthetic_obs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Synthetic observation of the 21cm line from numerical simulation

##Install SPARK package

In [1]:
import os
!mkdir rundir
os.chdir('/content/rundir/')
!rm -rf SPARK
!git clone https://github.com/GASKAP/SPARK
os.chdir('SPARK')
!pip install .

Cloning into 'SPARK'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 247 (delta 8), reused 4 (delta 4), pack-reused 232
Receiving objects: 100% (247/247), 13.84 MiB | 36.90 MiB/s, done.
Resolving deltas: 100% (119/119), done.
Processing /content/rundir/SPARK
  Preparing metadata (setup.py) ... done
  Created wheel for SPARK: filename=SPARK-0.1.0-py3-none-any.whl size=22018 sha256=f5b3ce1e7069544185913326059c038f5841089bab950113922595c7981f44ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-gc8bou23/wheels/a0/35/7b/555676e88c53cdc7f1bdd009c10952c9c089fe423c666cc79e
Successfully built SPARK


In [2]:
os.chdir('/content/')

##Import packages

In [3]:
import numpy as np
from astropy.io import fits
import astropy.table as pytabs
import matplotlib.pyplot as plt

from SPARK.synthetic import synth

##Generate synthetic observation

In [5]:
    # Open data
    path = '/content/rundir/SPARK/data/'

    hdu_list_rho = fits.open(path + 'rho_cube_sample.fits')
    hdu_list_T = fits.open(path + 'T_cube_sample.fits')
    hdu_list_vz = fits.open(path + 'vz_cube_sample.fits')

    #Velocity range and channel spacing
    vmin = -40 #km.s-1
    vmax = 40 #km.s-1
    dv = 0.8 #km.s-1

    rho_cube = hdu_list_rho[0].data #g.cm-3
    T_cube = hdu_list_T[0].data #K
    vz_cube = hdu_list_vz[0].data #cm.s-1

    dz=40/1024 #pc

    core = synth(rho=rho_cube, T=T_cube, vz=vz_cube, dz=dz)
    cube, tau = core.gen(vmin=vmin, vmax=vmax, dv=dv, thin=False)
    cube_thin, tau_thin = core.gen(vmin=vmin, vmax=vmax, dv=dv, thin=True)


100%|██████████| 101/101 [00:01<00:00, 62.94it/s]


##Select a range of kinetic temperature Tk - WNM

In [ ]:
cube_WNM, tau_WNM = core.gen(vmin=vmin, vmax=vmax, dv=dv, T_lim=[5000,np.inf], thin=True)
cube_LNM, tau_LNM = core.gen(vmin=vmin, vmax=vmax, dv=dv, T_lim=[500,5000], thin=True)
cube_CNM, tau_CNM = core.gen(vmin=vmin, vmax=vmax, dv=dv, T_lim=[0,500], thin=True)

##Plot synthetic obs
### Column density map

In [ ]:
NHI = np.sum(cube,0) * dv * core.C.value / 1.e19

#Plot integrated column density field TOT
fig = plt.figure(figsize=(8, 8))
ax = fig.add_axes([0.1,0.1,0.78,0.8])
ax.set_xlabel(r"x", fontsize=18.)
ax.set_ylabel(r"y", fontsize=18.)
img = ax.imshow(NHI, origin="lower")
colorbar_ax = fig.add_axes([0.89, 0.1, 0.02, 0.8])
cbar = fig.colorbar(img, cax=colorbar_ax, extend='both')
cbar.ax.tick_params(labelsize=14.)
cbar.set_label(r"N$_{HI}$ / [10$^{19}$ cm$^{-2}$]", fontsize=18.)
# plt.savefig("plot/" + 'NHI.png', format='png', bbox_inches='tight',
#             pad_inches=0.02)




In [ ]:
NHI_WNM = np.sum(cube_WNM,0) * dv * core.C.value / 1.e19

#Plot integrated column density field TOT
fig = plt.figure(figsize=(8, 8))
ax = fig.add_axes([0.1,0.1,0.78,0.8])
ax.set_xlabel(r"x", fontsize=18.)
ax.set_ylabel(r"y", fontsize=18.)
img = ax.imshow(NHI_WNM, origin="lower")
colorbar_ax = fig.add_axes([0.89, 0.1, 0.02, 0.8])
cbar = fig.colorbar(img, cax=colorbar_ax, extend='both')
cbar.ax.tick_params(labelsize=14.)
cbar.set_label(r"N$_{HI,WNM}$ / [10$^{19}$ cm$^{-2}$]", fontsize=18.)
# plt.savefig("plot/" + 'NHI.png', format='png', bbox_inches='tight',
#             pad_inches=0.02)

###Mosaic spectra

In [ ]:
#Velocity array
v = np.arange(vmin,vmax+dv, dv)

#Plot mosaic spectra
def norm(pval):
    return (pval - pmin) / float(pmax - pmin)

ny = 4; nx = 4
center_y = 16; center_x = 16
fig, axs = plt.subplots(4, 4, sharex=True, sharey=True, figsize=(14.,10.))
fig.subplots_adjust(hspace=0, wspace=0, left=0, right=1, top=1, bottom=0)
for i in np.arange(ny):
    for j in np.arange(nx):
        axs[i][j].set_xlim([-50,50])
        axs[i][j].plot(v, cube[:,center_y+i,center_x+j], color='orange',
                       linewidth=2., label="full")
        axs[i][j].plot(v, cube_thin[:,center_y+i,center_x+j], "--",
                       color='cornflowerblue', linewidth=2., label="thin")
        axs[i][j].plot(v, cube_WNM[:,center_y+i,center_x+j], "--",
                       color='r', linewidth=2., label="WNM thin")
        axs[i][j].plot(v, cube_LNM[:,center_y+i,center_x+j], "--",
                       color='g', linewidth=2., label="LNM thin")
        axs[i][j].plot(v, cube_CNM[:,center_y+i,center_x+j], "--",
                       color='b', linewidth=2., label="CNM thin")
        if j == 0: axs[i][j].set_ylabel(r'T$_b$ (k)', fontsize=16)
        axs[i][j].set_xlabel(r'v$_{LSR}$ (km s$^{-1}$)', fontsize=16)
plt.legend(loc = 1, numpoints = 1)
leg = plt.gca().get_legend()
ltext  = leg.get_texts()
plt.setp(ltext, fontsize = 'small')
# plt.savefig("plot/" + 'mosaic_spectra.png', format='png', bbox_inches='tight',
#             pad_inches=0.02)


In [ ]:
#Plot mosaic spectra tau
def norm(pval):
    return (pval - pmin) / float(pmax - pmin)

ny = 4; nx = 4
center_y = 16; center_x = 16
fig, axs = plt.subplots(4, 4, sharex=True, sharey=True, figsize=(14.,10.))
fig.subplots_adjust(hspace=0, wspace=0, left=0, right=1, top=1, bottom=0)
for i in np.arange(ny):
    for j in np.arange(nx):
        axs[i][j].set_xlim([-50,50])
        axs[i][j].plot(v, np.exp(-tau[:,center_y+i,center_x+j]), color='orange',
                       linewidth=2., label="full")
        axs[i][j].plot(v, np.exp(-tau_thin[:,center_y+i,center_x+j]), "--",
                       color='cornflowerblue', linewidth=2., label="thin")
        axs[i][j].plot(v, np.exp(-tau_WNM[:,center_y+i,center_x+j]), "--",
                       color='r', linewidth=2., label="WNM thin")
        axs[i][j].plot(v, np.exp(-tau_LNM[:,center_y+i,center_x+j]), "--",
                       color='g', linewidth=2., label="LNM thin")
        axs[i][j].plot(v, np.exp(-tau_CNM[:,center_y+i,center_x+j]), "--",
                       color='b', linewidth=2., label="CNM thin")
        if j == 0: axs[i][j].set_ylabel(r'e$^{- \tau}$', fontsize=16)
        axs[i][j].set_xlabel(r'v$_{LSR}$ (km s$^{-1}$)', fontsize=16)
plt.legend(loc = 1, numpoints = 1)
leg = plt.gca().get_legend()
ltext  = leg.get_texts()
plt.setp(ltext, fontsize = 'small')
# plt.savefig("plot/" + 'mosaic_spectra.png', format='png', bbox_inches='tight',
#             pad_inches=0.02)
